In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from model_clip_bert import MultimodalDataset, MultimodalClassifier, evaluate_model
from tqdm import tqdm
from transformers import get_scheduler


C:\Developpement\Cours\IFT714-NLP\Devoir2\.venv\ProjetMAMI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. Vérification du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [3]:
# Tokeniser
max_length = 128  # Longueur maximale des séquences tokenisées

# Capacité du modèle
batch_size = 16  # Taille des batchs
num_epochs = 1

# Paramètres du modèle
dropout = 0.5  # Dropout
learning_rate = 2e-5  # Taux d'apprentissage

In [4]:
# 3. Charger les données
train_df = pd.read_csv("../preprocessing_for_CLIP/train.csv")  # Doit contenir : file_name, text, label
test_texts = pd.read_csv("../preprocessing_for_CLIP/test_texts.csv")
test_labels = pd.read_csv("../preprocessing_for_CLIP/test_labels.csv")
test_df = pd.merge(test_texts, test_labels, on="file_name")

# 4. Séparation train/val
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42)

# 5. Créer les datasets
train_dataset = MultimodalDataset(train_data, image_dir="../Dataset/TRAINING", max_length=max_length)
val_dataset = MultimodalDataset(val_data, image_dir="../Dataset/TRAINING")
test_dataset = MultimodalDataset(test_df, image_dir="../Dataset/test")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

train_steps_per_epoch = len(train_loader)
total_steps = num_epochs * train_steps_per_epoch

In [5]:
# 6. Charger le modèle
model = MultimodalClassifier().to(device)

# 7. Définir l'optimizer et la loss
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

scheduler = get_scheduler(
    "linear",  # ou "cosine"
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

criterion = torch.nn.CrossEntropyLoss()

In [6]:
# 8. Entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        logits = model(
            text_input_ids=batch["text_input_ids"].to(device),
            text_attention_mask=batch["text_attention_mask"].to(device),
            image_pixel_values=batch["image_pixel_values"].to(device),
        )
        loss = criterion(logits, batch["label"].to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")

    val_acc, val_f1 = evaluate_model(model, val_loader, device)
    print(f"Validation Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}")

# 9. Test final
test_acc, test_f1 = evaluate_model(model, test_loader, device)
print("=== Test Results ===")
print(f"Accuracy: {test_acc:.4f}")
print(f"F1 Score : {test_f1:.4f}")

100%|██████████| 563/563 [04:01<00:00,  2.33it/s]


Epoch 1 - Loss: 0.4458
Validation Accuracy: 0.8360, F1 Score: 0.8441


100%|██████████| 563/563 [04:20<00:00,  2.16it/s]


Epoch 2 - Loss: 0.2319
Validation Accuracy: 0.8520, F1 Score: 0.8560
=== Test Results ===
Accuracy: 0.6850
F1 Score : 0.7424


In [7]:
# 6. Export des données

import csv
import os
import numpy as np

# Fonction pour formater les dictionnaires contenant des np.int64
def format_dict(d):
    return " , ".join(f"{int(k)} : {int(v)}" for k, v in d.items())

# Hyperparamètres et résultats
params = {
    "max_length": max_length,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "dropout": dropout,
    "val_acc": round(val_acc, 4),
    "val_f1_score": round(val_f1, 4),
    "test_acc": round(test_acc, 4),
    "test_f1_score": round(test_f1, 4),
}

# Nom du fichier CSV
output_file = "BERT+CLIP_hyperparameters_results.csv"

# Vérifier si le fichier existe déjà
if os.path.exists(output_file):
    # Lire le fichier existant
    with open(output_file, mode='r', newline='', encoding='utf-8-sig') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Déterminer le numéro de la prochaine colonne (valeurs_X)
    header = rows[0]
    existing_value_columns = [col for col in header[1:] if col.startswith("valeurs_")]
    if existing_value_columns:
        last_index = max(int(col.split("_")[1]) for col in existing_value_columns)
        new_col_name = f"valeurs_{last_index + 1}"
    else:
        new_col_name = "valeurs_1"

    # Ajouter le nom de la nouvelle colonne dans l'en-tête
    header.append(new_col_name)

    # Mettre à jour les lignes avec les nouvelles valeurs
    for i in range(1, len(rows)):  # On commence à 1 pour ignorer l'en-tête
        key = rows[i][0]
        if key in params:
            rows[i].append(params[key])  # Ajouter la valeur correspondante
        else:
            rows[i].append("")  # Ajouter une cellule vide pour les lignes sans correspondance

    # Écrire les nouvelles données dans le fichier
    with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

else:
    # Fichier n'existe pas encore, le créer et écrire les données
    with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)

        # Écrire l'en-tête
        writer.writerow(["Hyperparamètres et Résultats", "valeurs_1"])

        # Écrire les valeurs
        for key, value in params.items():
            writer.writerow([key, value])

print(f"Données exportées avec succès dans {output_file}!")

Données exportées avec succès dans BERT+CLIP_hyperparameters_results.csv!
